# LUNG CANCER DETECTION 

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import*
import os
import tensorflow as tf
import random as rn
from matplotlib import pyplot as plt
import itertools
from keras.models import Model
import matplotlib.pyplot as plt
from keras.optimizers import Adam
%matplotlib inline
import json

Using TensorFlow backend.


In [2]:
#CREATING VARIABLES AND ASSIGNING PATH
train_path='New_Segment/train'
valid_path='New_Segment/valid'
test_path='New_Segment/test'

In [24]:
train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['binary'],batch_size=10)
valid_batches=ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),classes=['binary'],batch_size=5)
test_batches=ImageDataGenerator().flow_from_directory(test_path,target_size=(224,224),classes=['binary'],batch_size=1)

Found 1082 images belonging to 2 classes.
Found 197 images belonging to 2 classes.
Found 1 images belonging to 1 classes.


In [30]:
import time

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)


In [31]:
time_callback = TimeHistory()

In [32]:
import json
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

In [33]:
from tensorflow.keras.applications.vgg16 import VGG16

In [34]:
from tensorflow.keras import layers 

In [36]:
base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [38]:
for layer in base_model.layers:
    layer.trainable = False

In [39]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(2, activation='softmax')(x)



In [40]:
model = tf.keras.models.Model(base_model.input, x)

In [41]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [42]:
model.compile(tf.keras.optimizers.Adam(lr=.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
learn_control = ReduceLROnPlateau(monitor='val_accuracy', patience=5,
                                  verbose=1,factor=0.2)

filepath="VGG_16_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history= model.fit_generator(train_batches, steps_per_epoch=108,
                              validation_data=valid_batches,validation_steps=39,epochs=25,verbose=1,
                              callbacks=[learn_control, checkpoint,time_callback]
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 108 steps, validate for 39 steps
Epoch 1/25
107/108 [============================>.] - ETA: 0s - loss: 2.5513 - accuracy: 0.7458
Epoch 00001: val_accuracy improved from -inf to 0.83077, saving model to VGG_16_weights.best.hdf5
108/108 [==============================] - 20s 187ms/step - loss: 2.5502 - accuracy: 0.7453 - val_loss: 1.6462 - val_accuracy: 0.8308
Epoch 2/25
107/108 [============================>.] - ETA: 0s - loss: 1.2649 - accuracy: 0.8644
Epoch 00002: val_accuracy did not improve from 0.83077
108/108 [==============================] - 17s 155ms/step - loss: 1.2550 - accuracy: 0.8647 - val_loss: 1.8585 - val_accuracy: 0.8154
Epoch 3/25
107/108 [============================>.] - ETA: 0s - loss: 0.7363 - accuracy: 0.9171
Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.

Epoch 00003: val_accuracy did not improve from 0.83077
108/108 [==============================] - 17s 156ms/step - l

In [44]:
time_callback